In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from tqdm.autonotebook import tqdm
import numpy as np
import faiss

/home/gero/.local/share/virtualenvs/vectorstorage-KAy25Pwz/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
train_data = pd.read_csv("train_data_categories.csv")
data = train_data[['video_id', 'title']]
tags_train = train_data["tags"]
taxonomy = pd.read_csv("IAB_tags.csv")

print(data.columns)
print(data.head(5))

print(taxonomy.head(5))
print(taxonomy.columns)

Index(['video_id', 'title'], dtype='object')
                           video_id  \
0  9007f33c8347924ffa12f922da2a179d   
1  9012707c45233bd601dead57bc9e2eca   
2  e01d6ebabbc27e323fa1b7c581e9b96a   
3  a00b145242be3ebc3b311455e94917af   
4  b01a682bf4dfcc09f1e8fac5bc18785a   

                                               title  
0  Пацанский клининг. Шоу «ЧистоТачка» | Повелите...  
1  СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...  
2  Злые языки | Выпуск 1, Сезон 1 | Непорочность ...  
3                 $1000 шоу | 1 выпуск | Автобоулинг  
4                    В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ  
  Уровень 1 (iab)         Уровень 2 (iab)      Уровень 3 (iab)
0       Транспорт                     NaN                  NaN
1       Транспорт  Типы кузова автомобиля                  NaN
2       Транспорт  Типы кузова автомобиля  Грузовой автомобиль
3       Транспорт  Типы кузова автомобиля                Седан
4       Транспорт  Типы кузова автомобиля            Универсал
Index(['Уров

In [39]:
tags_train \
    .apply(lambda x: str(x).split(", ")) \
    .apply(lambda x: len(x) if x != [","] else 0) \
    .value_counts()

tags
1    603
2    364
3     80
4      2
Name: count, dtype: int64

In [4]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence', )
dim = 768 # размер вектора эмбеддинга

No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.


#### Создаем эмбеддинги для названий видео

In [4]:
data['title'][0]

'Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17'

In [4]:
# data['title_vector'] = data['title'].apply(lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

#### Создаем векторы для тегов:
 Для каждого 1 уровня иерархии в отдельности и для следующих уровней формата уровень 1: уровень 2: уровень 3 

In [6]:
def get_tags():
    tags = {}
    for i, row in tqdm(taxonomy.iterrows()):
        if isinstance(row['Уровень 1 (iab)'], str):
            tags[row['Уровень 1 (iab)']] = model.encode(row['Уровень 1 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 2 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 3 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
    return tags

tags = get_tags()
tags_list = list(tags.keys())
vectors = np.array(list(tags.values()))

611it [00:23, 25.66it/s]


In [12]:
import json

def list_to_json(lst, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(lst, f, ensure_ascii=False)

list_to_json(tags_list, "tags_list.json")

In [15]:
# with open("tags_list.json", 'r', encoding='utf-8') as f:
#     tags_list2 = json.load(f)
# type(tags_list2)

list

### Создаем векторную базу faiss для эффективного векторного поиска

In [22]:
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
print(index.ntotal)
index.add(vectors)
print(index.ntotal)

0
610


In [23]:
faiss.write_index(index, "tags_index.ivf")

#### Смотрим несколько получившихся примеров 
Генерим по 3 близких предсказания для каждого названия видео

In [ ]:
topn = 3
scores, predictions = index.search(np.array(data['title_vector'].to_list()[:10]), topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags_list)[predictions[j]])
    print("SAMPLE", data['title'].to_list()[:10][j])
    print("\n")